In [ ]:
using ITensors,ITensorMPS
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")

In [ ]:
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

In [ ]:
include("overload_Spin=3h.jl")

In [ ]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for j=1:N-1
        ops += J,   "Sz",j, "Sz",j+1
        ops += J/2, "S+",j, "S-",j+1
        ops += J/2, "S-",j, "S+",j+1
    end
    #make it a ring
#    ops += J,   "Sz",N, "Sz",1
#    ops += J/2, "S+",N, "S-",1
#    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for j=1:N
        ops += -B, "Sz",j
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

In [ ]:
function cmag(wf::MPS,j_list)    
    psi = copy(wf)
    sites = siteinds(psi)

    for j in j_list
        if isodd(j)
            psi = apply( op("S+",sites[j]) * op("S-",sites[j+1]), psi )
        else
            psi = apply( op("S-",sites[j]) * op("S+",sites[j+1]), psi )
        end    
    end

    #----------------------------------
    return normalize!(psi)
end

In [ ]:
# one or two sites reduced density matrix
#--------------------------------------
function get_2pdm(wf::MPS,locs)
    ket = copy(wf)
    sites = siteinds(ket)
    n_locs = length(locs)

    if n_locs==1
        A = locs[1]
        orthogonalize!(ket,A)
        rho = prime(ket[A],sites[A]) * dag(ket[A])

    elseif n_locs==2
        A,B = locs
        orthogonalize!(ket,A)
        bra = prime(dag(ket),linkinds(ket))
        
        rho = prime(ket[A],linkinds(ket,A-1)) * prime(bra[A],sites[A])
        [ rho *= ket[j]*bra[j] for j in A+1:B-1 ]
        rho *= prime(ket[B],linkinds(ket,B)) * prime(bra[B],sites[B])  
    
    end

    #-----------------------------------
    return rho
end

In [ ]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log2(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


In [ ]:
#   entanglement details between two sites
#-----------------------------------
function calc_Neg(wf::MPS,locs)
    psi = copy(wf)
    A,B = locs

    rho_AB = get_2pdm(psi,[A,B])
    rho_AB_PT = calc_PartialTranspose(rho_AB)
    egn_val = calc_Eigenvalues(rho_AB_PT)

    neg_entries =  [ lam for lam in egn_val if lam<0 ]
    Neg = abs(sum(neg_entries))

    #-----------------------------------
    return Neg
end


# Partial between two sites (test program)
#   takes a partial transpose over 2nd site
#------------------------------------------------

function calc_PartialTranspose(density_matrix)
    rho = copy(density_matrix)

    oldinds = inds(rho)
    newinds = [oldinds[1],oldinds[2],oldinds[4],oldinds[3]]
    rho_PT = swapinds(rho,oldinds,newinds)
    
    #-----------------------------
    return rho_PT
end


#   entanglement details between two sites
#-----------------------------------
function calc_MutInf(wf::MPS,locs)
    psi = copy(wf)
    A,B = locs

    rho_A  = get_2pdm(psi,[A,A])
    rho_B  = get_2pdm(psi,[B,B])
    rho_AB = get_2pdm(psi,[A,B])

    S_A = calc_SvN(rho_A)
    S_B = calc_SvN(rho_B)
    S_AB = calc_SvN(rho_AB)

    MutInf = S_A + S_B - S_AB

    #-----------------------------------
    return MutInf
end




In [ ]:
#define the parameters of the hamiltonian
#----------------------------------------------------------------------------

J = 1
B = 0.1

nsweeps = 11
maxdim = [10,25,50,100,150,200,250]
cutoff = [1E-10]

In [ ]:
N = 20
#--------------------------------------------

site_list = collect(1:N) 
sites = siteinds( j->isodd(j) ? "S=1/2" : "S=3/2", N ) 

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true)
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

In [ ]:
exp_H = inner(psi0',H,psi0) ;
exp_Sz = expect( psi0, "Sz" ) ;
net_Sz = sum(exp_Sz) ;

println("energy/N = $(exp_H/N)")
println("net(Sz)/N = $(net_Sz/N)")

states = [ isodd(j) ? "Dn" : "+3/2" for j in 1:N]
psi_Neel = productMPS(sites,states)
alpha_Neel = inner(psi0,psi_Neel)
#alpha_Neel = 0.99053*exp(-0.03547*N)

In [ ]:
rho = get_4pdm(psi0,[2,6])
#inds(rho)

#rho = get_2pdm(psi0,[2,7])
#wcalc_properties(rho)

In [ ]:
psi = cmag(psi0,[3,5,7,9,13,15,17])

plot( expect(psi,"Sz"),marker=:circle )